In [36]:
import pandas as pd
import numpy as np
df = pd.read_csv('internship_assignment.csv')


df = df.rename(columns={'user_id_hashed': 'user_id'})

# Map the hashed user IDs to new user IDs
user_id_dict = {hashed_id: new_id for new_id, hashed_id in enumerate(df['user_id'].unique(), start=1)}
df['user_id'] = df['user_id'].map(user_id_dict)

# Filter the data for the first user
first_user_id = df['user_id'].iloc[0]
U1 = df[df['user_id'] == first_user_id]
U1 = U1.sort_values('dt')

U1['dt'] = pd.to_datetime(U1['dt'])





U1.tail()



,user_id,dt,learning_goal,selected_track_id,selected_project_id,selected_project,topic,project,project_difficulty,step_id,step,step_difficulty,action
27619,1,2023-03-12 19:26:48,Kotlin Developer,3,106,Sorting Tool (Kotlin),NaN,NaN,NaN,0,NaN,NaN,selected_track
20924,1,2023-03-12 19:26:50,Kotlin Developer,3,106,Sorting Tool (Kotlin),NaN,NaN,NaN,0,NaN,NaN,changed_track
27620,1,2023-03-12 19:27:54,Android Development with Kotlin,16,106,Sorting Tool (Kotlin),NaN,NaN,NaN,0,NaN,NaN,selected_track
20925,1,2023-03-12 19:27:55,Android Development with Kotlin,16,0,NaN,NaN,NaN,NaN,0,NaN,NaN,changed_track
20907,1,2023-03-12 19:27:55,Android Development with Kotlin,16,0,NaN,NaN,Sorting Tool (Kotlin),NaN,9628,Sorting Tool (Kotlin) → X-files,hard,changed_project


In [35]:
# Resample the data to one-second frequency
U1 = U1.resample('S').sum()

# Use asfreq() to keep missing rows
U1 = U1.asfreq()

# Set the datetime index to a range of timestamps
U1.index = pd.date_range(start=U1.index.min(), end=U1.index.max(), freq='S')

# Compute the rolling mean and standard deviation for each column
U1_mean = U1.rolling(window=60).mean()
U1_std = U1.rolling(window=60).std()

# Create a new dataframe with the mean and standard deviation for each column
U1_ts = pd.concat([U1_mean, U1_std], axis=1)
U1_ts.columns = ['learning_goal_mean', 'selected_track_id_mean', 'selected_project_id_mean', 'selected_project_mean', 'topic_mean', 'project_mean', 'project_difficulty_mean', 'step_id_mean', 'step_mean', 'step_difficulty_mean', 'action_mean', 'learning_goal_std', 'selected_track_id_std', 'selected_project_id_std', 'selected_project_std', 'topic_std', 'project_std', 'project_difficulty_std', 'step_id_std', 'step_std', 'step_difficulty_std', 'action_std']

# Fill missing values for each column separately
U1_ts.fillna(method='ffill', inplace=True)
U1_ts.fillna(method='bfill', inplace=True)

# Add a column for the target variable (the next action)
U1_ts['target'] = U1_ts['action_mean'].shift(-5)

# Drop the last 5 rows, which have missing target values
U1_ts = U1_ts.iloc[:-5]

# Print the resulting dataframe
print(U1_ts.head())



TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'